<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp/blob/main/phase2/course1/week4/Exercise_4_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import os
import zipfile

In [11]:
!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

data_dir = '/tmp/h-or-s'

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall(data_dir)
zip_ref.close()

--2021-01-01 12:30:04--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.128.128, 74.125.124.128, 172.217.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.128.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2021-01-01 12:30:04 (159 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [27]:
class TrainCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.999):
      print("\n정확도가 99.9%를 넘겼으므로 학습을 중단합니다.")
      self.model.stop_training = True

callbacks = TrainCallback()

In [23]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(150,150,3)),
  tf.keras.layers.MaxPool2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPool2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 148, 148, 128)     3584      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 74, 74, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 72, 72, 64)        73792     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 82944)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2654240   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [31]:
train_datagen = ImageDataGenerator(rescale=1/.255)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(150,150),
    batch_size=10,
    class_mode='binary'
)

Found 80 images belonging to 2 classes.


In [32]:
result = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=20,
    verbose=1,
    callbacks=[callbacks]
)

Epoch 1/20
8/8 [==============================] - 7s 880ms/step - loss: 0.3232 - accuracy: 0.8375
Epoch 2/20
8/8 [==============================] - 7s 883ms/step - loss: 0.3118 - accuracy: 0.8750
Epoch 3/20
8/8 [==============================] - 7s 870ms/step - loss: 0.2774 - accuracy: 0.8375
Epoch 4/20
8/8 [==============================] - 7s 870ms/step - loss: 0.2412 - accuracy: 0.8750
Epoch 5/20
8/8 [==============================] - 7s 876ms/step - loss: 0.1653 - accuracy: 0.9250
Epoch 6/20
8/8 [==============================] - 7s 871ms/step - loss: 0.1198 - accuracy: 1.0000

정확도가 99.9%를 넘겼으므로 학습을 중단합니다.
